<a href="https://colab.research.google.com/github/dolmani38/QA/blob/main/SQuAD2.0_code_test_0513.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Squad2.0 학습 코드 테스트  (영어)

https://github.com/huggingface/datasets

https://huggingface.co/transformers/custom_datasets.html#qa-squad



In [1]:

if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

     |████████████████████████████████| 225kB 5.1MB/s 
     |████████████████████████████████| 245kB 8.7MB/s 
     |████████████████████████████████| 112kB 9.2MB/s 


In [3]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric, load_from_disk

# Print all the available datasets
print(list_datasets())

# Load a dataset and print the first example in the training set
squad_dataset = load_dataset('squad_v2') #,cache_dir ='/content/drive/MyDrive/korQuAD2.1/dataset')


['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews', 'allocine', 'alt', 'amazon_polarity', 'amazon_reviews_multi', 'amazon_us_reviews', 'ambig_qa', 'amttl', 'anli', 'app_reviews', 'aqua_rat', 'aquamuse', 'ar_cov19', 'ar_res_reviews', 'ar_sarcasm', 'arabic_billion_words', 'arabic_pos_dialect', 'arabic_speech_corpus', 'arcd', 'arsentd_lev', 'art', 'arxiv_dataset', 'ascent_kb', 'aslg_pc12', 'asnq', 'asset', 'assin', 'assin2', 'atomic', 'autshumato', 'babi_qa', 'banking77', 'bbaw_egyptian', 'bbc_hindi_nli', 'bc2gm_corpus', 'best2009', 'bianet', 'bible_para', 'big_patent', 'billsum', 'bing_coronavirus_query_set', 'biomrc', 'blended_skill_talk', 'blimp', 'blog_authorship_corpus', 'bn_hate_speech', 'bookcorpus', 'bookcorpusopen', 'boolq', 'bprec', 'break_data', 'brwac', 'bsd_ja_en', 'bswac', 'c3', 'c4', 'cail2018', 'caner', 'capes', 'catalonia_independence', 'cawac', 'cbt', 'c

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/ba48bc29b974701e9ba8d80ac94f3e3df924aba41b764dcf9851debea7c672e4. Subsequent calls will reuse this data.


In [4]:
squad_dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [5]:
from datasets import Dataset
import pandas as pd

squad_dataset_train = {}
squad_dataset_train['id'] = []
squad_dataset_train['title'] = []
squad_dataset_train['context'] = []
squad_dataset_train['question'] = []
squad_dataset_train['answers'] = []

# 정답은 없는 것은 날려 버림...
for sdt in squad_dataset['train']:
    if  len(sdt['answers']['answer_start']) > 0:
        squad_dataset_train['id'].append(sdt['id'])
        squad_dataset_train['title'].append(sdt['title'])
        squad_dataset_train['context'].append(sdt['context'])
        squad_dataset_train['question'].append(sdt['question'])
        squad_dataset_train['answers'].append(sdt['answers'])

dataset_train = Dataset.from_dict(squad_dataset_train)     


In [6]:
len(dataset_train)

86821

In [7]:
squad_dataset_val = {}
squad_dataset_val['id'] = []
squad_dataset_val['title'] = []
squad_dataset_val['context'] = []
squad_dataset_val['question'] = []
squad_dataset_val['answers'] = []

for sdt in squad_dataset['validation']:
    if  len(sdt['answers']['answer_start']) > 0:
        squad_dataset_val['id'].append(sdt['id'])
        squad_dataset_val['title'].append(sdt['title'])
        squad_dataset_val['context'].append(sdt['context'])
        squad_dataset_val['question'].append(sdt['question'])
        squad_dataset_val['answers'].append(sdt['answers'])

dataset_val = Dataset.from_dict(squad_dataset_val)  

In [8]:
len(dataset_val)

5928

In [9]:
!pip install transformers

     |████████████████████████████████| 2.3MB 6.4MB/s 
     |████████████████████████████████| 3.3MB 29.9MB/s 
     |████████████████████████████████| 901kB 51.8MB/s 


In [10]:
train_contexts = dataset_train['context']
train_questions = dataset_train['question']
train_answers = dataset_train['answers']

In [11]:
val_contexts = dataset_val['context']
val_questions = dataset_val['question']
val_answers = dataset_val['answers']

In [12]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        try:
        
            gold_text = answer['text']
            if isinstance(gold_text,list):
                gold_text = gold_text[0]

            start_idx = answer['answer_start']
            if isinstance(start_idx,list):
                start_idx = start_idx[0]

            end_idx = start_idx + len(gold_text)

            # sometimes squad answers are off by a character or two – fix this
            if context[start_idx:end_idx] == gold_text:
                answer['answer_end'] = [end_idx]
            elif context[start_idx-1:end_idx-1] == gold_text:
                answer['answer_start'] = [start_idx - 1]
                answer['answer_end'] = [end_idx - 1]     # When the gold label is off by one character
            elif context[start_idx-2:end_idx-2] == gold_text:
                answer['answer_start'] = [start_idx - 2]
                answer['answer_end'] = [end_idx - 2]     # When the gold label is off by two characters
        except Exception as e:
            pass
            #print('context',context)
            #print('gold_text',gold_text)


add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [13]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [14]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'][0]))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'][0] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [15]:
len(train_encodings[1])

512

In [16]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [17]:
len(train_dataset)

86821

In [18]:
from transformers import DistilBertForQuestionAnswering

model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [19]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)


In [20]:
len(train_loader)

5427

In [21]:

# Print iterations progress
class ProgressBar:

  def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
    self.total = total
    self.prefix = prefix
    self.suffix = suffix
    self.decimals = decimals
    self.length = length
    self.fill = fill
    self.printEnd = printEnd
    self.ite = 0

  def printProgress(self,iteration, text):
      self.ite += iteration
      percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

      filledLength = int(self.length * self.ite // self.total)
      bar = self.fill * filledLength + '-' * (self.length - filledLength)
      print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
      # Print New Line on Complete
      if self.ite == self.total: 
          print()

In [33]:
import random

def seeding(seed):

    SEED = seed

    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

In [22]:
model = DistilBertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/squad2.0/model')


In [ ]:
import matplotlib.pyplot as plt

batch_count = len(train_loader)
model.to(device)
model.train()
#loss_graph = []
epochs = 5
for epoch in range(epochs):
    seeding(1024+epoch)
    pb = ProgressBar(total=batch_count,prefix='Epoch '+ str(epoch+1) + '/' + str(epochs))
    current_batch = 0
    for batch in train_loader:
        current_batch+=1
        seeding(1024+epoch+current_batch)
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        #print('input_ids',len(input_ids))
        attention_mask = batch['attention_mask'].to(device)
        #print('attention_mask',len(attention_mask))
        start_positions = batch['start_positions'].to(device)
        #print('start_positions',len(start_positions))
        end_positions = batch['end_positions'].to(device)
        #print('end_positions',len(end_positions))
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        pb.printProgress(+1,'loss:' + str(loss))
        #loss_graph.append(loss.item())
    model.save_pretrained('/content/drive/MyDrive/squad2.0/model')
model.eval()


'''
plt.figure(figsize=(12, 6))
plt.plot(loss_graph,label='loss')
plt.legend()
plt.show()
'''

Epoch 1/5 |███-----------------| 15.2%   loss:tensor(0.2111, device='cuda:0', grad_fn=<DivBackward0>)

In [23]:
examples = dataset_val

In [24]:
examples[0]['answers']['text']

['France', 'France', 'France', 'France']

In [25]:
def display_example(example):    
    from pprint import pprint

    #idx = qid_to_example_index[qid]
    q = example['question']
    c = example['context']
    a = example['answers']['text']
    
    print(f'Example {example["id"]} of {len(examples)}\n---------------------')
    print(f"Q: {q}\n")
    print("Context:")
    pprint(c)
    print(f"\nTrue Answers:\n{a}")

In [26]:
display_example(examples[0])

Example 56ddde6b9a695914005b9628 of 5928
---------------------
Q: In what country is Normandy located?

Context:
('The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the '
 'people who in the 10th and 11th centuries gave their name to Normandy, a '
 'region in France. They were descended from Norse ("Norman" comes from '
 '"Norseman") raiders and pirates from Denmark, Iceland and Norway who, under '
 'their leader Rollo, agreed to swear fealty to King Charles III of West '
 'Francia. Through generations of assimilation and mixing with the native '
 'Frankish and Roman-Gaulish populations, their descendants would gradually '
 'merge with the Carolingian-based cultures of West Francia. The distinct '
 'cultural and ethnic identity of the Normans emerged initially in the first '
 'half of the 10th century, and it continued to evolve over the succeeding '
 'centuries.')

True Answers:
['France', 'France', 'France', 'France']


In [40]:
from pprint import pprint

model.to(device)

def get_prediction(example,display=False):

    if display:
        display_example(example)
    # given a question id (qas_id or qid), load the example, get the model outputs and generate an answer
    question = example['question'] #examples[qid_to_example_index[qid]].question_text
    context = example['context'] #examples[qid_to_example_index[qid]].context_text

    #print(f"Q: {question}\n")
    #print("Context:")
    #pprint(context)

    inputs = tokenizer.encode_plus(question, context, max_length=512,return_tensors='pt')

    inputs.to(device)
    
    outputs = model(**inputs)

    #print(outputs)
    #print(outputs)    
    answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(outputs[1]) + 1 

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    if display:
        print('')
        print('Predict answer:',answer)
    return answer

In [41]:
prediction = get_prediction(examples[0],display=True)


Example 56ddde6b9a695914005b9628 of 5928
---------------------
Q: In what country is Normandy located?

Context:
('The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the '
 'people who in the 10th and 11th centuries gave their name to Normandy, a '
 'region in France. They were descended from Norse ("Norman" comes from '
 '"Norseman") raiders and pirates from Denmark, Iceland and Norway who, under '
 'their leader Rollo, agreed to swear fealty to King Charles III of West '
 'Francia. Through generations of assimilation and mixing with the native '
 'Frankish and Roman-Gaulish populations, their descendants would gradually '
 'merge with the Carolingian-based cultures of West Francia. The distinct '
 'cultural and ethnic identity of the Normans emerged initially in the first '
 'half of the 10th century, and it continued to evolve over the succeeding '
 'centuries.')

True Answers:
['France', 'France', 'France', 'France']

Predict answer: france


In [29]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = example['answers']['text'] #[answer["text"] for answer in example['answers'] if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example - 
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]
        
    return gold_answers

In [30]:
def evaluate_example(example,display=False):
   
    gold_answers = get_gold_answers(example)
    prediction = get_prediction(example)
    em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
    f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)
    if display:
        print(f"Question: {example['question']}")
        print(f"Prediction: {prediction}")
        print(f"True Answers: {gold_answers}")
        print(f"EM: {em_score} \t F1: {f1_score}")
    return em_score,f1_score

In [31]:
em,f1 = evaluate_example(examples[0],display=True)

Question: In what country is Normandy located?
Prediction: france
True Answers: ['France', 'France', 'France', 'France']
EM: 1 	 F1: 1.0


In [42]:
import numpy as np

ems = []
f1s = []
pb = ProgressBar(total=len(examples),prefix='Evaluate... ')
for example in examples:
    em,f1 = evaluate_example(example)
    ems.append(em)
    f1s.append(f1)
    pb.printProgress(+1,'EM:' + str(em) + '  F1:' + str(f1))
print('EM:',np.mean(ems))
print('F1:',np.mean(f1s))

Evaluate...  |████████████████████| 100.0%   EM:0  F1:0
EM: 0.27614709851551955
F1: 0.3673101687870961


https://rajpurkar.github.io/SQuAD-explorer/

<iframe src="https://rajpurkar.github.io/SQuAD-explorer/"></iframe>

